In [47]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

!pip install albumentations

from PIL import Image
import cv2
import albumentations as A

import time
import os
from tqdm.notebook import tqdm

!pip install -q segmentation-models-pytorch
!pip install -q torchsummary

from torchsummary import summary
import segmentation_models_pytorch as smp

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from torchvision.transforms import ToTensor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
print(torchvision.__version__)

0.16.0+cpu


In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMAGE_PATH = 'dataset/semantic_drone_dataset/original_images/'
MASK_PATH = 'dataset/semantic_drone_dataset/label_images_semantic/'

batch_size = 8
n_classes = 23


In [50]:
IMAGE_PATH

'dataset/semantic_drone_dataset/original_images/'

In [51]:
n_classes = 23 

def create_df():
    name = []
    for dirname, _, filenames in os.walk(IMAGE_PATH):
        for filename in filenames:
            name.append(filename.split('.')[0])
    
    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

def get_mask(name):
    mask_path = MASK_PATH + name
    mask = Image.open(mask_path).convert('RGB')
    mask = np.array(mask)
    mask = np.argmax(mask, axis=2)
    mask = np.expand_dims(mask, axis=2)
    mask = torch.Tensor(mask).long()
    return mask

def get_transforms():
    transform = {
        'train': A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.Transpose(p=0.5),
            A.Normalize(),
            ToTensor(),
        ]),
        'valid': A.Compose([
            A.Normalize(),
            ToTensor(),
        ]),
    }

    return transform

In [52]:

df = create_df()
print('Total Images: ', len(df))

Total Images:  400


In [53]:

class DroneDataset(Dataset):
    def __init__(self, df, image_path, mask_path, transform=None):
        self.df = df
        self.image_path = image_path
        self.mask_path = mask_path
        self.transform = transform
  
    def __len__(self):
        return len(self.df)
  
    def __getitem__(self, index):
        name = self.df['name'][index]
        image_path = self.image_path + name
        image = Image.open(image_path).convert('RGB')
        mask = get_mask(name)
      
        if self.transform:
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image = augmented['image']
            mask = augmented['mask']

        return image, mask


In [54]:

def train():
    train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)
  
    train_dataset = DroneDataset(train_df, IMAGE_PATH, MASK_PATH, get_transforms()['train'])
    valid_dataset = DroneDataset(valid_df, IMAGE_PATH, MASK_PATH, get_transforms()['valid'])
  
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)#до этого нумлодерс было 4
    valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0)#до этого нумлодерс было 4
  
    model = smp.Unet('resnet34', encoder_weights='imagenet', classes=n_classes).to(device)
  
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)
  
    num_epochs = 10
  
    train_loss_history = []
    valid_loss_history = []
    mIoU_history = []
  
    best_mIoU = 0.0
  
    for epoch in range(num_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        mIoU = 0.0
      
        model.train()
        for images, masks in tqdm(train_loader):
            images = images.to(device)
            masks = masks.to(device)
          
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks.squeeze(1))
            loss.backward()
            optimizer.step()
          
            train_loss += loss.item() * images.size(0)
        
        model.eval()
        with torch.no_grad():
            for images, masks in tqdm(valid_loader):
                images = images.to(device)
                masks = masks.to(device)
              
                outputs = model(images)
                loss = criterion(outputs, masks.squeeze(1))
                valid_loss += loss.item() * images.size(0)
              
                outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
                masks = masks.squeeze(1).detach().cpu().numpy()
                mIoU += compute_iou(outputs, masks)
          
        train_loss = train_loss / len(train_loader.dataset)
        valid_loss = valid_loss / len(valid_loader.dataset)
        mIoU = mIoU / len(valid_loader.dataset)
        
        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)
        mIoU_history.append(mIoU)
      
        print(f'Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Valid Loss: {valid_loss:.6f}, mIoU: {mIoU:.6f}')
      
        if mIoU > best_mIoU:
            best_mIoU = mIoU
            torch.save(model.state_dict(), 'best_model.pth')
      
        scheduler.step(valid_loss)
      
    return train_loss_history, valid_loss_history, mIoU_history


In [55]:
train_loss_history, valid_loss_history, mIoU_history = train()

  0%|          | 0/40 [00:00<?, ?it/s]

KeyError: 'name'